# Đề thi Giữa kỳ Hệ khuyến nghị 2025 (đề thi mẫu).

## Khoa học dữ liệu IUH. Thời gian làm bài: 90 phút.

**Bài 1.** (*2.5 điểm*) Vấn đề về *cold-start*.

Cho file dữ liệu csv về 20 bài hát được 100 users rate trong tháng qua, điểm từ 1 đến 5. Cấu trúc file: *id user, id bài hát, điểm rating*.

1) Sử dụng kỹ thuật tổng hợp 2 thông tin: số rating kết hợp rating trung bình, hãy đề xuất ra top 5 bài hát nên nghe cho một user cold-start.
2) Giả sử có một user đã nghe bài hát STT 1. Sử dụng kỹ thuật đếm số cặp bài hát được nghe bởi cùng user, hãy đề xuất ra top 5 bài hát mà user này có thể cũng muốn nghe theo xu hướng.
3) Hỏi nếu có thêm 1 item mới đưa vào hệ thống thì ta nên xử lý thế nào? SV tham khảo thêm tại đây:

https://en.wikipedia.org/wiki/Cold_start_(recommender_systems)

lab 1, 2 có nói 

In [2]:
import pandas as pd

# 1️⃣ Đọc dữ liệu
df = pd.read_csv("ratings.csv")   # file gồm: user_id, song_id, rating
print(df.head())

# 2️⃣ Tính trung bình và số lượng rating cho mỗi bài hát
song_stats = df.groupby('movieId').agg(
    num_ratings=('rating', 'count'),
    avg_rating=('rating', 'mean')
).reset_index()

# 3️⃣ Tính các giá trị trung gian cho công thức IMDb
C = song_stats['avg_rating'].mean()        # trung bình rating toàn bộ bài hát
m = song_stats['num_ratings'].quantile(0.75)  # ngưỡng 75% số lượng rating

# 4️⃣ Tính điểm weighted score
def weighted_rating(x, m=m, C=C):
    v = x['num_ratings']
    R = x['avg_rating']
    return (v / (v + m)) * R + (m / (v + m)) * C

song_stats['score'] = song_stats.apply(weighted_rating, axis=1)

# 5️⃣ Sắp xếp theo điểm giảm dần và chọn top 5
top_5 = song_stats.sort_values(by='score', ascending=False).head(5)

print("\n🎧 Top 5 bài hát nên nghe cho user cold-start:")
print(top_5[['movieId', 'avg_rating', 'num_ratings', 'score']])


   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

🎧 Top 5 bài hát nên nghe cho user cold-start:
      movieId  avg_rating  num_ratings     score
277       318    4.429022          317  4.396816
659       858    4.289062          192  4.243095
2224     2959    4.272936          218  4.232872
224       260    4.231076          251  4.197546
46         50    4.237745          204  4.196535


In [6]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

# 1️⃣ Đọc dữ liệu
df = pd.read_csv("bai1_ratings.csv")   # gồm: user_id, song_id, rating

# 2️⃣ Xây dựng ma trận co-occurrence (đếm số lần 2 bài hát được nghe cùng user)
cooccur = defaultdict(int)

# Duyệt qua từng user
for user, group in df.groupby('user_id'):
    songs = group['song_id'].unique()
    # Tạo tất cả các cặp bài hát mà user nghe
    for i, j in combinations(sorted(songs), 2):
        cooccur[(i, j)] += 1
        cooccur[(j, i)] += 1   # song song, đảm bảo tính đối xứng

# 3️⃣ Tạo DataFrame từ dict
cooccur_df = pd.DataFrame([
    {'song_i': i, 'song_j': j, 'count': c}
    for (i, j), c in cooccur.items()
])

# 4️⃣ Chọn ra các bài hát đồng xuất hiện với bài hát có ID = 1
target_song = 1
related_songs = cooccur_df[cooccur_df['song_i'] == target_song].sort_values('count', ascending=False)

# 5️⃣ Lấy top 5 bài hát đồng nghe nhiều nhất
top5_related = related_songs.head(5)

print("\n🎧 Top 5 bài hát thường được nghe cùng với bài hát STT 1:")
print(top5_related)



🎧 Top 5 bài hát thường được nghe cùng với bài hát STT 1:
     song_i  song_j  count
2         1       6     24
4         1       9     17
194       1       7     15
56        1      11     15
90        1      13     15


🎯 1. Bản chất của vấn đề item cold-start

Bài hát mới → chưa có dữ liệu rating hoặc người nghe nào liên quan.

Do đó, các phương pháp dựa vào lịch sử như Collaborative Filtering (CF) không thể hoạt động được,
vì CF cần mối liên hệ giữa item–user (tức là ma trận rating phải có dữ liệu).

🧩 2. Các cách xử lý điển hình (theo lý thuyết và tài liệu Wikipedia)
🔹 (a) Dựa trên thông tin nội dung (Content-based Filtering)

Sử dụng các đặc trưng (features) của bài hát mới để gợi ý:

Ví dụ: thể loại (genre), ca sĩ, nhịp điệu (tempo), độ dài, năm phát hành, mô tả bài hát...

Ta tìm những bài hát có nội dung tương tự (content similarity) và gợi ý cho người dùng đã thích các bài đó.

📘 Ví dụ:
Bài hát mới thuộc thể loại pop, do Taylor Swift hát → ta có thể gợi ý nó cho người dùng từng nghe nhiều bài pop hoặc bài Taylor Swift khác.

➡️ Ưu điểm: xử lý được khi item hoàn toàn mới.
➡️ Nhược điểm: cần metadata phong phú (đôi khi không có).

🔹 (b) Kết hợp (Hybrid Approach)

Kết hợp CF + Content-based để tận dụng ưu điểm của cả hai.

Ban đầu dùng content-based để gợi ý tạm thời cho item mới.

Khi bài hát đó đã có vài rating hoặc người nghe, chuyển dần sang CF để cập nhật xu hướng thực tế.

📘 Ví dụ: Netflix hoặc Spotify thường khởi đầu bằng metadata (thể loại, diễn viên, ca sĩ, tags),
sau đó học dần từ hành vi người dùng thực tế.

🔹 (c) Khởi tạo bằng chuyên gia hoặc quảng bá

Khi item mới được đưa lên, có thể đẩy tạm vào top trend, mục “mới phát hành” hoặc được quảng bá để thu hút lượt nghe đầu tiên.

Sau khi có đủ dữ liệu, hệ thống recommender bắt đầu học.

🔹 (d) Sử dụng mô hình sinh embedding (Deep Learning / Transfer Learning)

Áp dụng các mô hình như Word2Vec, Doc2Vec, hoặc Transformer encoder để sinh embedding cho item mới dựa trên nội dung mô tả (lyrics, metadata...).

Dùng embedding đó để tính độ tương đồng với các item khác mà không cần rating ban đầu.

**Bài 2.** (*3 điểm*) Vấn đề xây dựng *content-based data*.

Sử dụng dataset movie_lens trong đó có file *movies.csv* mô tả tên films và các thể loại (một phim có thể thuộc nhiều thể loại).

1. Đọc dữ liệu và lọc lại 100 films có nhiều thể loại nhất.
2. Xét một film X trong đó, tính ra top 10 films tương đồng với film X này nhất bằng độ tương đồng Jaccard.
3. Cho một user Y đã xem được 5 bộ films $1,2,3,4,5$ trong danh sách 100 films đã liệt kê ở trên, hãy xây dựng một profile cơ bản cho user này và khuyến nghị ra top 10 films mà user Y này có thể muốn xem.

In [7]:
import pandas as pd

# Đọc dữ liệu movies.csv (file có sẵn trong MovieLens)
movies = pd.read_csv("movies.csv")
print(movies.head())

# Tách các thể loại
movies['genre_list'] = movies['genres'].apply(lambda x: x.split('|') if isinstance(x, str) else [])

# Đếm số lượng thể loại
movies['num_genres'] = movies['genre_list'].apply(len)

# Lọc top 100 phim có nhiều thể loại nhất
top100 = movies.sort_values(by='num_genres', ascending=False).head(100).reset_index(drop=True)
print("\n🎬 Top 5 trong 100 phim có nhiều thể loại nhất:")
print(top100[['movieId', 'title', 'genres', 'num_genres']].head())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

🎬 Top 5 trong 100 phim có nhiều thể loại nhất:
   movieId                                              title  \
0    81132                                      Rubber (2010)   
1    26701  Patlabor: The Movie (Kidô keisatsu patorebâ: T...   
2    56152                                   Enchanted (2007)   
3     2987                    Who Framed Roger Rabbit? (1988)   
4    32031                      

In [8]:
# Hàm tính Jaccard similarity giữa 2 danh sách thể loại
def jaccard_similarity(genres_a, genres_b):
    set_a, set_b = set(genres_a), set(genres_b)
    if not set_a or not set_b:
        return 0
    return len(set_a & set_b) / len(set_a | set_b)

# Chọn 1 phim X (ví dụ phim đầu tiên)
film_X = top100.iloc[0]
print("\n🎞 Phim X:", film_X['title'], "— Thể loại:", film_X['genre_list'])

# Tính độ tương đồng Jaccard với các phim khác
top100['similarity'] = top100['genre_list'].apply(
    lambda g: jaccard_similarity(film_X['genre_list'], g)
)

# Lọc ra top 10 phim tương đồng nhất (bỏ chính nó)
similar_films = top100[top100['movieId'] != film_X['movieId']].sort_values(
    by='similarity', ascending=False
).head(10)

print("\n🎯 Top 10 phim tương đồng nhất với", film_X['title'])
print(similar_films[['title', 'genres', 'similarity']])



🎞 Phim X: Rubber (2010) — Thể loại: ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Film-Noir', 'Horror', 'Mystery', 'Thriller', 'Western']

🎯 Top 10 phim tương đồng nhất với Rubber (2010)
                                                title  \
20          Sherlock Holmes: A Game of Shadows (2011)   
53                Seven-Per-Cent Solution, The (1976)   
67               Ichi the Killer (Koroshiya 1) (2001)   
1   Patlabor: The Movie (Kidô keisatsu patorebâ: T...   
90                             Negotiator, The (1998)   
41                                Strange Days (1995)   
55                                  Chase, The (1994)   
56              Dragonheart 2: A New Beginning (2000)   
37                               Blood Diamond (2006)   
57                                  Mummy, The (1999)   

                                               genres  similarity  
20     Action|Adventure|Comedy|Crime|Mystery|Thriller    0.600000  
53      Adventure|Comedy|Crime|Drama|Myst

In [12]:
top100[top100['title'].str.contains("Toy Story", case=False)]


,movieId,title,genres,genre_list,num_genres,similarity,user_sim
70,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,"[Adventure, Animation, Children, Comedy, Fanta...",6,0.142857,0.352941


In [13]:
film_title = "Toy Story 3 (2010)"

# Lọc ra dòng phim có tên khớp (dòng đầu tiên nếu có trùng)
film_X = top100[top100['title'] == film_title].iloc[0]

print("\n🎞 Phim X:", film_X['title'])
print("Thể loại:", film_X['genre_list'])

# Tính độ tương đồng Jaccard với các phim khác
top100['similarity'] = top100['genre_list'].apply(
    lambda g: jaccard_similarity(film_X['genre_list'], g)
)

# Lọc ra top 10 phim tương đồng nhất (bỏ chính nó)
similar_films = top100[top100['movieId'] != film_X['movieId']].sort_values(
    by='similarity', ascending=False
).head(10)

print("\n🎯 Top 10 phim tương đồng nhất với", film_X['title'])
print(similar_films[['title', 'genres', 'similarity']])



🎞 Phim X: Toy Story 3 (2010)
Thể loại: ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'IMAX']

🎯 Top 10 phim tương đồng nhất với Toy Story 3 (2010)
                                                title  \
43                              Ant Bully, The (2006)   
73  Shrek Forever After (a.k.a. Shrek: The Final C...   
4                                       Robots (2005)   
80                    How to Train Your Dragon (2010)   
14  Twelve Tasks of Asterix, The (Les douze travau...   
25                 Madagascar: Escape 2 Africa (2008)   
46                                        Home (2015)   
47                             Kung Fu Panda 2 (2011)   
50                                  Inside Out (2015)   
52      Puss in Boots (Nagagutsu o haita neko) (1969)   

                                               genres  similarity  
43   Adventure|Animation|Children|Comedy|Fantasy|IMAX    1.000000  
73   Adventure|Animation|Children|Comedy|Fantasy|IMAX    1.000000  
4   Ad

In [16]:
film_id = 78499  # Ví dụ chọn movieId = 1

# Lọc ra phim tương ứng
film_X = top100[top100['movieId'] == film_id].iloc[0]

print("\n🎞 Phim X:", film_X['title'])
print("Thể loại:", film_X['genre_list'])

# Tính độ tương đồng Jaccard với các phim khác
top100['similarity'] = top100['genre_list'].apply(
    lambda g: jaccard_similarity(film_X['genre_list'], g)
)

# Lọc ra top 10 phim tương đồng nhất (bỏ chính nó)
similar_films = top100[top100['movieId'] != film_X['movieId']].sort_values(
    by='similarity', ascending=False
).head(10)

print("\n🎯 Top 10 phim tương đồng nhất với", film_X['title'])
print(similar_films[['title', 'genres', 'similarity']])



🎞 Phim X: Toy Story 3 (2010)
Thể loại: ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'IMAX']

🎯 Top 10 phim tương đồng nhất với Toy Story 3 (2010)
                                                title  \
43                              Ant Bully, The (2006)   
73  Shrek Forever After (a.k.a. Shrek: The Final C...   
4                                       Robots (2005)   
80                    How to Train Your Dragon (2010)   
14  Twelve Tasks of Asterix, The (Les douze travau...   
25                 Madagascar: Escape 2 Africa (2008)   
46                                        Home (2015)   
47                             Kung Fu Panda 2 (2011)   
50                                  Inside Out (2015)   
52      Puss in Boots (Nagagutsu o haita neko) (1969)   

                                               genres  similarity  
43   Adventure|Animation|Children|Comedy|Fantasy|IMAX    1.000000  
73   Adventure|Animation|Children|Comedy|Fantasy|IMAX    1.000000  
4   Ad

# 2.3

In [10]:
# Giả sử user Y đã xem 5 phim đầu tiên trong danh sách
watched_ids = top100.iloc[[0,1,2,3,4]]['movieId'].tolist()
watched_genres = top100[top100['movieId'].isin(watched_ids)]['genre_list']

# Tạo profile người dùng (tập hợp các thể loại yêu thích)
user_profile = set(g for sublist in watched_genres for g in sublist)
print("\n🧍‍♂️ User profile:", user_profile)

# Tính độ tương đồng Jaccard giữa user profile và từng phim
def user_similarity(user_genres, movie_genres):
    if not movie_genres:
        return 0
    return len(set(user_genres) & set(movie_genres)) / len(set(user_genres) | set(movie_genres))

top100['user_sim'] = top100['genre_list'].apply(lambda g: user_similarity(user_profile, g))

# Gợi ý 10 phim chưa xem có độ tương đồng cao nhất
recommendations = top100[~top100['movieId'].isin(watched_ids)].sort_values(
    by='user_sim', ascending=False
).head(10)

print("\n🎬 Top 10 phim được khuyến nghị cho user Y:")
print(recommendations[['title', 'genres', 'user_sim']])



🧍‍♂️ User profile: {'Drama', 'Mystery', 'Thriller', 'Film-Noir', 'Horror', 'Action', 'Comedy', 'Animation', 'Sci-Fi', 'Western', 'Children', 'Musical', 'Romance', 'Adventure', 'IMAX', 'Fantasy', 'Crime'}

🎬 Top 10 phim được khuyến nghị cho user Y:
                                                title  \
5                                      Tangled (2010)   
7                                Osmosis Jones (2001)   
8                                Interstate 60 (2002)   
9                                        Mulan (1998)   
10                             Mars Needs Moms (2011)   
11  Aqua Teen Hunger Force Colon Movie Film for Th...   
12          Aelita: The Queen of Mars (Aelita) (1924)   
13                                   Inception (2010)   
6                                        Pulse (2006)   
55                                  Chase, The (1994)   

                                               genres  user_sim  
5   Animation|Children|Comedy|Fantasy|Musical|Roma...  0.

**Bài 3.** (*2 điểm*) Vấn đề dự đoán *missing value* trong lọc cộng tác.

Cho một bảng *utility matrix* giữa 4 user và 4 item như bên dưới:

|Rating     | $I_1$    | $I_2$  | $I_3$   |  $I_4$  |
|----------|----------|--------|---------|---------|
| $U_1$    | 4     | 3     | ?    |   4 | 
| $U_2$ | 3     | 3      | 5     |   5    |
| $U_3$  | 5     | 4      |   3  |    4 |
| $U_4$|   ?  |   2    | 3 |   5|

1. Dự đoán hai missing rating $r_{U_1, I_3}$ và $r_{U_4, I_1}$ bằng cách dùng độ tương đồng Pearson.
2. Câu hỏi tương tự trên nhưng chuẩn hoá bảng theo dạng center và sau đó dùng độ tương đồng cosine. So sánh các kết quả thu được.

In [17]:
import pandas as pd
import numpy as np

# Tạo bảng utility matrix
data = {
    'I1': [4, 3, 5, np.nan],
    'I2': [3, 3, 4, 2],
    'I3': [np.nan, 5, 3, 3],
    'I4': [4, 5, 4, 5]
}
ratings = pd.DataFrame(data, index=['U1','U2','U3','U4'])
print("🔹 Utility matrix:\n", ratings, "\n")

# --- Hàm tính Pearson similarity giữa hai user ---
def pearson_sim(u1, u2):
    common = u1.dropna().index.intersection(u2.dropna().index)
    if len(common) < 2:  # cần ít nhất 2 điểm chung
        return 0
    u1_r = u1[common] - u1[common].mean()
    u2_r = u2[common] - u2[common].mean()
    num = np.sum(u1_r * u2_r)
    den = np.sqrt(np.sum(u1_r**2)) * np.sqrt(np.sum(u2_r**2))
    return num/den if den != 0 else 0

# --- Ma trận similarity ---
users = ratings.index
sim_matrix = pd.DataFrame(np.zeros((4,4)), index=users, columns=users)

for u in users:
    for v in users:
        if u != v:
            sim_matrix.loc[u,v] = pearson_sim(ratings.loc[u], ratings.loc[v])

print("🔹 Ma trận độ tương đồng Pearson:\n", sim_matrix, "\n")

# --- Hàm dự đoán rating ---
def predict_rating(user, item):
    user_mean = ratings.loc[user].mean(skipna=True)
    sims, num, den = [], 0, 0
    for other in users:
        if other != user and not np.isnan(ratings.loc[other, item]):
            sim = sim_matrix.loc[user, other]
            sims.append(sim)
            num += sim * (ratings.loc[other, item] - ratings.loc[other].mean(skipna=True))
            den += abs(sim)
    return user_mean + num / den if den != 0 else user_mean

# Dự đoán 2 giá trị missing
r_U1_I3 = predict_rating('U1', 'I3')
r_U4_I1 = predict_rating('U4', 'I1')

print(f"🎯 Dự đoán r_U1,I3 = {r_U1_I3:.2f}")
print(f"🎯 Dự đoán r_U4,I1 = {r_U4_I1:.2f}")


🔹 Utility matrix:
      I1  I2   I3  I4
U1  4.0   3  NaN   4
U2  3.0   3  5.0   5
U3  5.0   4  3.0   4
U4  NaN   2  3.0   5 

🔹 Ma trận độ tương đồng Pearson:
      U1        U2        U3        U4
U1  0.0  0.500000  0.500000  1.000000
U2  0.5  0.000000 -0.707107  0.755929
U3  0.5 -0.707107  0.000000  0.188982
U4  1.0  0.755929  0.188982  0.000000 

🎯 Dự đoán r_U1,I3 = 3.50
🎯 Dự đoán r_U4,I1 = 3.21


In [18]:
# --- Mean-center dữ liệu ---
ratings_centered = ratings.sub(ratings.mean(axis=1), axis=0)

# --- Cosine similarity ---
def cosine_sim(u1, u2):
    common = u1.dropna().index.intersection(u2.dropna().index)
    if len(common) == 0:
        return 0
    u1_vec, u2_vec = u1[common], u2[common]
    num = np.nansum(u1_vec * u2_vec)
    den = np.sqrt(np.nansum(u1_vec**2)) * np.sqrt(np.nansum(u2_vec**2))
    return num/den if den != 0 else 0

cosine_sim_matrix = pd.DataFrame(np.zeros((4,4)), index=users, columns=users)
for u in users:
    for v in users:
        if u != v:
            cosine_sim_matrix.loc[u,v] = cosine_sim(ratings_centered.loc[u], ratings_centered.loc[v])

print("\n🔹 Ma trận độ tương đồng Cosine (mean-centered):\n", cosine_sim_matrix, "\n")

# --- Dự đoán với cosine similarity ---
def predict_cosine(user, item):
    user_mean = ratings.loc[user].mean(skipna=True)
    num, den = 0, 0
    for other in users:
        if other != user and not np.isnan(ratings_centered.loc[other, item]):
            sim = cosine_sim_matrix.loc[user, other]
            num += sim * ratings_centered.loc[other, item]
            den += abs(sim)
    return user_mean + num/den if den != 0 else user_mean

r_U1_I3_cos = predict_cosine('U1', 'I3')
r_U4_I1_cos = predict_cosine('U4', 'I1')

print(f"🎯 Dự đoán (Cosine) r_U1,I3 = {r_U1_I3_cos:.2f}")
print(f"🎯 Dự đoán (Cosine) r_U4,I1 = {r_U4_I1_cos:.2f}")



🔹 Ma trận độ tương đồng Cosine (mean-centered):
           U1        U2        U3        U4
U1  0.000000  0.471405  0.408248  0.907959
U2  0.471405  0.000000 -0.707107  0.712697
U3  0.408248 -0.707107  0.000000  0.154303
U4  0.907959  0.712697  0.154303  0.000000 

🎯 Dự đoán (Cosine) r_U1,I3 = 3.53
🎯 Dự đoán (Cosine) r_U4,I1 = 3.19


**Bài 4.** (*2.5 điểm*) Vấn đề thuật toán lọc cộng tác ở dạng *user-based* và *item-based*.

Sử dụng lại dataset ở bài 1 hoặc dataset *movie_lens* và chọn một thuật toán *collaborative filtering* đã chuẩn bị sẵn, thực hiện hai yêu cầu:
1) In ra ma trận tương đồng giữa các user và giữa các item sử dụng *sklearn.metrics.pairwise*.
2) Duyệt theo từng user và cho biết sẽ khuyến nghị item nào cho họ.
3) Duyệt theo từng item và cho biết sẽ khuyến nghị item này cho các user nào. Có nhận xét gì về kết quả thu được ở 2) và 3)?

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Đọc dữ liệu (dùng file bài 1)
df = pd.read_csv("bai1_ratings.csv")
print(df.head())

# Pivot để tạo ma trận user-item (users x songs)
ratings_matrix = df.pivot_table(index='user_id', columns='song_id', values='rating')

# Điền giá trị thiếu bằng 0 (người chưa nghe bài hát)
ratings_filled = ratings_matrix.fillna(0)

print("\n🔹 Ma trận user-item (hiển thị 5 dòng đầu):")
print(ratings_filled.head())


   user_id  song_id  rating
0        1        1       2
1        1       14       4
2        1        9       5
3        1        2       1
4        1       16       4

🔹 Ma trận user-item (hiển thị 5 dòng đầu):
song_id   1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
user_id                                                                         
1        2.0  1.0  0.0  0.0  0.0  2.0  0.0  0.0  5.0  0.0  0.0  4.0  0.0  4.0   
2        2.0  1.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0   
3        0.0  0.0  0.0  0.0  4.0  0.0  0.0  0.0  2.0  2.0  0.0  0.0  0.0  2.0   
4        1.0  3.0  5.0  0.0  1.0  5.0  0.0  0.0  0.0  4.0  1.0  0.0  1.0  0.0   
5        0.0  0.0  0.0  3.0  0.0  1.0  0.0  0.0  0.0  0.0  3.0  0.0  5.0  0.0   

song_id   15   16   17   18   19   20  
user_id                                
1        0.0  4.0  0.0  0.0  0.0  5.0  
2        0.0  0.0  0.0  0.0  4.0  0.0  
3        0.0  2.0  0.0  0.0  0.0  0.0  
4        0.0  0.0  3

🔹 2️⃣ Tính ma trận tương đồng
🧠 a. Giữa user–user

In [20]:
user_sim = cosine_similarity(ratings_filled)
user_sim_df = pd.DataFrame(user_sim, index=ratings_filled.index, columns=ratings_filled.index)

print("\n🔹 Ma trận tương đồng giữa các user:")
print(user_sim_df.round(2).head())



🔹 Ma trận tương đồng giữa các user:
user_id   1     2     3     4     5     6     7     8     9     10   ...  \
user_id                                                              ...   
1        1.00  0.20  0.44  0.20  0.29  0.34  0.22  0.23  0.46  0.18  ...   
2        0.20  1.00  0.00  0.45  0.18  0.37  0.22  0.29  0.28  0.19  ...   
3        0.44  0.00  1.00  0.22  0.00  0.00  0.18  0.19  0.56  0.11  ...   
4        0.20  0.45  0.22  1.00  0.21  0.42  0.34  0.17  0.50  0.52  ...   
5        0.29  0.18  0.00  0.21  1.00  0.76  0.23  0.44  0.35  0.34  ...   

user_id   91    92    93    94    95    96    97    98    99    100  
user_id                                                              
1        0.03  0.43  0.56  0.13  0.15  0.77  0.43  0.61  0.26  0.01  
2        0.13  0.00  0.54  0.37  0.58  0.19  0.42  0.14  0.67  0.14  
3        0.56  0.14  0.13  0.00  0.12  0.63  0.19  0.18  0.09  0.21  
4        0.21  0.08  0.57  0.29  0.72  0.28  0.25  0.47  0.41  0.44  
5        0

🧠 b. Giữa item–item

In [21]:
item_sim = cosine_similarity(ratings_filled.T)
item_sim_df = pd.DataFrame(item_sim, index=ratings_filled.columns, columns=ratings_filled.columns)

print("\n🔹 Ma trận tương đồng giữa các item (bài hát):")
print(item_sim_df.round(2).head())



🔹 Ma trận tương đồng giữa các item (bài hát):
song_id    1     2     3     4     5     6     7     8     9     10    11  \
song_id                                                                     
1        1.00  0.22  0.25  0.24  0.21  0.51  0.26  0.24  0.48  0.29  0.27   
2        0.22  1.00  0.33  0.25  0.16  0.53  0.25  0.42  0.33  0.46  0.44   
3        0.25  0.33  1.00  0.36  0.14  0.24  0.30  0.30  0.28  0.41  0.16   
4        0.24  0.25  0.36  1.00  0.14  0.28  0.35  0.34  0.31  0.26  0.23   
5        0.21  0.16  0.14  0.14  1.00  0.19  0.28  0.24  0.30  0.34  0.25   

song_id    12    13    14    15    16    17    18    19    20  
song_id                                                        
1        0.34  0.28  0.19  0.23  0.30  0.36  0.32  0.40  0.25  
2        0.26  0.35  0.23  0.17  0.31  0.28  0.19  0.25  0.34  
3        0.19  0.38  0.26  0.29  0.20  0.34  0.31  0.36  0.23  
4        0.23  0.28  0.18  0.34  0.30  0.18  0.25  0.21  0.30  
5        0.35  0.21  0.18  0.

🔹 3️⃣ User-based Collaborative Filtering

Ý tưởng:
Gợi ý cho mỗi user những item mà người dùng tương tự đã đánh giá cao nhưng user đó chưa nghe

In [22]:
def recommend_user_based(user_id, n_recommend=5):
    user_index = ratings_filled.index.get_loc(user_id)
    sim_scores = user_sim[user_index]
    
    # Tính điểm dự đoán cho các item chưa xem
    user_ratings = ratings_filled.iloc[user_index]
    unrated_items = user_ratings[user_ratings == 0].index
    scores = {}
    for item in unrated_items:
        # Lấy các user khác đã rating item này
        item_ratings = ratings_filled[item]
        mask = item_ratings > 0
        if mask.sum() == 0:
            continue
        # Dự đoán dựa vào weighted average
        num = np.dot(sim_scores[mask], item_ratings[mask])
        den = np.sum(sim_scores[mask])
        scores[item] = num/den if den != 0 else 0

    # Trả về top n item có điểm dự đoán cao nhất
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:n_recommend]

# Thử gợi ý cho 1 user cụ thể (ví dụ user 1)
print("\n🎯 Gợi ý cho user 1 (user-based CF):")
print(recommend_user_based(1, n_recommend=5))



🎯 Gợi ý cho user 1 (user-based CF):
[(8, 2.997014084702143), (17, 2.9683107586830264), (11, 2.928379932791757), (19, 2.9102166000820326), (5, 2.853453324342054)]


🔹 4️⃣ Item-based Collaborative Filtering

Ý tưởng:
Gợi ý các user nên nghe bài hát tương tự với các bài họ đã nghe và thích.

In [23]:
def recommend_item_based(user_id, n_recommend=5):
    user_ratings = ratings_filled.loc[user_id]
    scores = {}
    for item in ratings_filled.columns:
        if user_ratings[item] > 0:
            continue  # bỏ qua bài đã nghe
        # Lấy các item tương tự với item hiện tại mà user đã nghe
        sim_items = item_sim_df[item]
        rated_items = user_ratings[user_ratings > 0].index
        num = np.dot(sim_items[rated_items], user_ratings[rated_items])
        den = np.sum(np.abs(sim_items[rated_items]))
        scores[item] = num/den if den != 0 else 0
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:n_recommend]

# Thử gợi ý cho user 1 theo item-based CF
print("\n🎯 Gợi ý cho user 1 (item-based CF):")
print(recommend_item_based(1, n_recommend=5))



🎯 Gợi ý cho user 1 (item-based CF):
[(5, 3.593399442741836), (15, 3.5656591339864283), (4, 3.4387807230861784), (7, 3.426116463389561), (17, 3.3975871904328954)]


🔹 5️⃣ Gợi ý theo từng user và từng item
🎧 a. Duyệt tất cả user

In [24]:
print("\n🔹 Gợi ý item cho từng user (user-based CF):")
for u in ratings_filled.index[:5]:  # in thử 5 user đầu
    recs = recommend_user_based(u, n_recommend=3)
    print(f"User {u} → gợi ý các bài hát: { [r[0] for r in recs] }")



🔹 Gợi ý item cho từng user (user-based CF):
User 1 → gợi ý các bài hát: [8, 17, 11]
User 2 → gợi ý các bài hát: [13, 10, 3]
User 3 → gợi ý các bài hát: [8, 12, 6]
User 4 → gợi ý các bài hát: [8, 19, 16]
User 5 → gợi ý các bài hát: [8, 14, 17]


🎵 b. Duyệt tất cả item (gợi ý người dùng phù hợp)

In [25]:
print("\n🔹 Gợi ý user cho từng item (item-based CF đảo chiều):")
for i in ratings_filled.columns[:5]:  # 5 item đầu
    # tìm user có nhiều bài tương tự với i
    sim_scores = item_sim_df[i].sort_values(ascending=False)[1:6]
    print(f"Item {i} tương tự với các item {list(sim_scores.index)} → nên gợi ý cho user từng nghe chúng.")



🔹 Gợi ý user cho từng item (item-based CF đảo chiều):
Item 1 tương tự với các item [6, 9, 19, 17, 12] → nên gợi ý cho user từng nghe chúng.
Item 2 tương tự với các item [6, 10, 11, 8, 13] → nên gợi ý cho user từng nghe chúng.
Item 3 tương tự với các item [10, 13, 19, 4, 17] → nên gợi ý cho user từng nghe chúng.
Item 4 tương tự với các item [3, 7, 8, 15, 9] → nên gợi ý cho user từng nghe chúng.
Item 5 tương tự với các item [16, 12, 17, 10, 9] → nên gợi ý cho user từng nghe chúng.


## yêu cầu thêm
### chuẩn bị sẵn 1 thuật toán lọc cộng tác + dùng kNN để khuyến nghị theo hướng user và hướng item nha (xem trong các bài lab hoặc lấy trên mạng đều được)

In [26]:
!pip install scikit-surprise


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-win_amd64.whl size=1290294 sha256=98513a76c77c6408d4424e6de7ef207415ed44ac0127e5fd117491dddcb9d25d
  Stored in directory: c:\users\anhtu\appdata\local\pip\cache\wheels\75\fa\bc\739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [27]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
import pandas as pd


In [28]:
# Đọc dữ liệu ratings (user_id, song_id, rating)
df = pd.read_csv("bai1_ratings.csv")
print(df.head())

# Chuẩn hóa dữ liệu cho thư viện surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'song_id', 'rating']], reader)

# Chia tập train/test
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


   user_id  song_id  rating
0        1        1       2
1        1       14       4
2        1        9       5
3        1        2       1
4        1       16       4


In [29]:
# Cấu hình KNN theo hướng user
sim_options = {
    'name': 'cosine',   # có thể thay bằng 'pearson'
    'user_based': True  # True = user-based
}

# Tạo mô hình
user_cf = KNNBasic(k=5, sim_options=sim_options)

# Huấn luyện
user_cf.fit(trainset)

# Dự đoán
predictions_user = user_cf.test(testset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [30]:
# Cấu hình theo hướng item
sim_options = {
    'name': 'cosine',
    'user_based': False  # False = item-based
}

item_cf = KNNBasic(k=5, sim_options=sim_options)
item_cf.fit(trainset)
predictions_item = item_cf.test(testset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [31]:
# Hàm khuyến nghị top-N item cho một user
def recommend_top_n(model, user_id, n=5):
    # Lấy tất cả item
    all_items = df['song_id'].unique()
    rated_items = df[df['user_id'] == user_id]['song_id'].unique()
    not_rated = [i for i in all_items if i not in rated_items]

    # Dự đoán rating cho các item chưa đánh giá
    predictions = [model.predict(user_id, iid) for iid in not_rated]
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    return [(pred.iid, round(pred.est, 2)) for pred in top_n]

# Ví dụ gợi ý cho user 1
print("\n🎯 Gợi ý cho user 1 (user-based CF):")
print(recommend_top_n(user_cf, 1))

print("\n🎯 Gợi ý cho user 1 (item-based CF):")
print(recommend_top_n(item_cf, 1))



🎯 Gợi ý cho user 1 (user-based CF):
[(11, 3.8), (7, 3.4), (5, 3.2), (3, 3.0), (8, 3.0)]

🎯 Gợi ý cho user 1 (item-based CF):
[(11, 3.43), (3, 3.37), (8, 3.36), (5, 3.32), (7, 3.3)]
